In [ ]:
%pip install --quiet --upgrade pip 
%pip install numpy --quiet
%pip install Pandas --quiet
%pip install sklearn --quiet
%pip install ipywidgets --quiet

# Horse Racing Results Predictor #

The American professional gambler [Bill Benter](https://en.wikipedia.org/wiki/Bill_Benter) is said to have made earned nearly $1 billion through the development of one of the most successful analysis computer software programs in the horse racing market.

Bill published his techniques in the paper [Computer-Based Horse Race Handicapping and Wagering Systems](https://www.gwern.net/docs/statistics/decision/1994-benter.pdf). 

The [YouTube Video by Ken Jee](https://www.youtube.com/watch?v=KEeUR8UDy-s) outlines how he did it, how difficult it was, and discusses whether it is likely to be able to replicate this feat today (hint: Ken thinks it highly unlikely for a number of reasons).

Inspired by video, this notebook examines the possibility of replicating Bill's success using data from modern day UK races.

**NOTE: This is a fun examination of the technique the can be used in predicting races. It is not intended to be accurate or valid. The author accepts no responsibility for the correctness, completeness or quality of the information provided. Please do not use this information to place any real-world bets. Gambling odds are always skewed in favour of the bookmaker and you will lose in the long run.**

In [ ]:
import numpy as np
import pandas as pd
import math
from abc import ABC, abstractmethod
from datetime import datetime, date

### Step 1: Load in the historic race data and ignore any horse that didn't complete the race

In [ ]:
import glob
from datetime import timedelta
results_files = glob.glob('Results_*.csv')
results_files

history = pd.concat([pd.read_csv(f) for f in results_files])
history = history[history['ResultStatus'] == 'CompletedRace']
history['Off'] =  pd.to_datetime(history['Off'], format='%m/%d/%Y %H:%M:%S')
history['Wins'] = history.apply(lambda r: 1 if r['FinishingPosition'] == 1 else 0, axis=1)

### Analyse factors to understand if they have influence on the outcome of races. 

Bill Benter suggested the following attributes:

Current condition:
- performance in recent races
- time since last race
- recent workout data
- age of horse

Past performance:
- finishing position in past races
- lengths behind winner in past races
- normalized times of past races

Adjustments to past performance:
- strength of competition in past races
- weight carried in past races
- jockey's contribution to past performances
- compensation for bad luck in past races
- compensation for advantageous or disadvantageous post position in past races

Present race situational factors:
- weight to be carried
- today's jockey's ability
- advantages or disadvantages of the assigned post position

Preferences which could influence the horse's performance in today's race:
- distance preference
- surface preference (turf vs dirt)
- condition of surface preference (wet vs dry)
- specific track preference

In [ ]:
history.columns

In [ ]:
races = history[['RaceId', 'CourseId', 'RaceType', 'Off', 'DecimalOdds', 'OfficialRating', 'RacingPostRating', 'TopSpeedRating',
       'DistanceInMeters', 'Going', 'Surface', 'HorseId', 'HorseName', 'JockeyId', 'JockeyName',
       'Age', 'HeadGear', 'RaceCardNumber','StallNumber', 'WeightInPounds', 
       'FinishingPosition','OverallBeatenDistance', 'RaceTimeInSeconds', 'Wins']].copy()

### Expand categorical variables

In [ ]:
surface_categories = ["Surface_AllWeather", "Surface_Dirt", "Surface_Turf"]
races = races.drop(surface_categories + ["Surface_Unknown"], axis=1, errors='ignore')
races["SurfaceTemp"] = races["Surface"]
races = pd.get_dummies(races, prefix="Surface", columns=["SurfaceTemp"], dtype=float)
races = races.drop("Surface_Unknown", axis=1) # Drop unknown surface as only small number.
races[surface_categories].value_counts()

In [ ]:
# Normalise going based on rules here: https://www.racingpost.com/guide-to-racing/what-is-the-going-ann7h6W6VB3b/
# Values should be: Firm, Good_To_Firm, Good, Good_To_Soft, Soft, Heavy
norm_map = ({
    "Good": "Good", 
    "Standard": "Good",
    "Soft": "Soft",
    "Good To Soft": "Good_To_Soft",
    "Good To Firm": "Good_To_Firm",
    "Heavy": "Heavy",
    "Good To Yielding": "Good_To_Soft",    
    "Yielding": "Good_To_Soft",
    "Standard To Slow": "Good_To_Soft",  
    "Very Soft": "Heavy",
    "Fast": "Firm",
    "Firm": "Firm",
    "Soft To Heavy": "Heavy",    
    "Yielding To Soft": "Soft",
    "Slow": "Soft",
    "Sloppy": "Heavy",
    "Muddy": "Heavy",
    "Frozen": "Heavy"
})

races["NormGoing"] = races["Going"].map(norm_map)


In [ ]:
going_categories  = ["Going_Good", "Going_Good_To_Soft", "Going_Soft", "Going_Good_To_Firm", "Going_Firm", "Going_Heavy"]
races = races.drop(going_categories, axis=1, errors='ignore')
races = pd.get_dummies(races, prefix="Going", columns=["NormGoing"], dtype=float)
races[going_categories].value_counts()

In [ ]:
race_type_categories = ["RaceType_Other", "RaceType_Hurdle", "RaceType_SteepleChase", "RaceType_Flat"]
races = races.drop(race_type_categories, axis=1, errors='ignore')
races["RaceTypeTemp"] = races["RaceType"]
races = pd.get_dummies(races, prefix="RaceType", columns=["RaceTypeTemp"], dtype=float)
races[race_type_categories].value_counts()

In [ ]:
races["DistanceInMeters"].hist(bins=30)

#### Calculate Horse Stats

Calculate the speed of each horse in the race

In [ ]:
races["Speed"] = races["DistanceInMeters"] / races["RaceTimeInSeconds"]

# Deal with invalid data (usually invalid race time) - fastest horse (over short distance) 55mps or approx 25 m/s
races.loc[races["Speed"] > 25, "Speed"] = np.nan

races.plot.scatter(x="DistanceInMeters", y="Speed")

In [ ]:
# Deal with invalid data - occasionally weight will be undefined (usually zero)
races.loc[races["WeightInPounds"] < 10, "WeightInPounds"] = np.nan

Calculate the number of horse in each race

In [ ]:
groups = races.groupby(['RaceId']).apply(lambda g: pd.Series({'HorseCount': g['RaceId'].count()}, index=['HorseCount']))
races = pd.merge(races, groups, how='left', on=['RaceId'])

Define feature factory abstraction that calculates feature values based on previous history (either in total or over a time window)

In [ ]:
from ipywidgets import IntProgress
from IPython.display import display

class RaceDataProcessor(ABC):

    def before_process_data(self, df: pd.DataFrame) -> None:
        # Update the underlying DataFrame before processing to e.g. set default values for new features
        pass

    def update(self, df : pd.DataFrame, history : pd.DataFrame, daily_slice : pd.DataFrame) -> None:
        # Update the processor with data
        pass


    def process_race_data(self, df : pd.DataFrame) -> None:
        df_start = df['Off'].min().date()
        slice_start = df_start + timedelta(days=1)
        df_end = df['Off'].max().date() + timedelta(days=1)
        
        days = (df_end - slice_start).days
        f = IntProgress(min=0, max=days) # instantiate the bar
        display(f)

        self.before_process_data(df)
        while slice_start < df_end:
            slice_end = slice_start + timedelta(days=1)
            history = df[df['Off'].dt.date < slice_start]
            daily_slice = df[(df['Off'].dt.date >= slice_start) & (df['Off'].dt.date < slice_end)]
            self.update(df, history, daily_slice)
            slice_start = slice_end
            f.value += 1


Calculate, for each race, whether the horse and jockey are known (i.e. have previously been involved in a race)

In [ ]:
class CalculateRacesWithKnownHorsesAndJockeys(RaceDataProcessor):
    def before_process_data(self, df: pd.DataFrame) -> None:
        df["KnownHorseAndJockey"] = False

    def update(self, df : pd.DataFrame, history : pd.DataFrame, daily_slice : pd.DataFrame) -> None:
        known_jockeys = history['JockeyId'].unique().tolist()
        known_runners = history['HorseId'].unique().tolist()
        temp = daily_slice.groupby('RaceId').apply(lambda g: self.__calculate_counts_for_race_group(g, known_jockeys, known_runners))
        temp = temp[(temp['HorseCount'] == temp['KnownJockeyCount']) & (temp['HorseCount'] == temp['KnownHorseCount'])]
        races_with_known_horses_and_jockeys = temp.reset_index()['RaceId'].unique().tolist()
        df.loc[df['RaceId'].isin(races_with_known_horses_and_jockeys), "KnownHorseAndJockey"] = True

    def __calculate_counts_for_race_group(self, race_group, known_jockeys, known_runners) -> pd.Series:
        new_columns = {'HorseCount': race_group['HorseId'].count()}
        new_columns['KnownHorseCount'] = race_group[race_group['HorseId'].isin(known_runners)]['HorseId'].count()
        new_columns['KnownJockeyCount'] = race_group[race_group['JockeyId'].isin(known_jockeys)]['JockeyId'].count()
        return pd.Series(new_columns, index=['HorseCount', 'KnownHorseCount', 'KnownJockeyCount']) 


In [ ]:
CalculateRacesWithKnownHorsesAndJockeys().process_race_data(races)

Calculate stats for each horse in the given daily slice based on the history up until that point

In [ ]:
class CalculateHorsesStats(RaceDataProcessor):
    NUMBER_OF_PRIOR_RACES = "NumberOfPriorRaces"
    LAST_RACE_GOING = "LastRaceGoing"
    LAST_RACE_SURFACE = "LastRaceSurface"
    LAST_RACE_DISTANCE = "LastRaceDistanceInMeters"
    LAST_RACE_WEIGHT = "LastRaceWeightInPounds"
    LAST_RACE_SPEED = "LastRaceSpeed"
    DAYS_REST_SINCE_LAST_RACE = "DaysRested"
    LAST_RACE_ODDS = "LastRaceDecimalOdds"
    LAST_RACE_OFFICIAL_RATING = "LastRaceOfficialRating"
    LAST_RACE_RACING_POST_RATING = "LastRaceRacingPostRating"
    LAST_RACE_TOP_SPEED_RATING = "LastRaceTopSpeedRating"
    AVG_RELATIVE_FINISHING_POSITION = "LastRaceAvgRelFinishingPosition"
    ONE_DAY = np.timedelta64(1, 'D')

    def before_process_data(self, df: pd.DataFrame) -> None:
        df.loc[:, self.NUMBER_OF_PRIOR_RACES] = 1
        df.loc[:, self.LAST_RACE_GOING] = np.nan
        self.new_column_names = ([
            self.NUMBER_OF_PRIOR_RACES,
            self.LAST_RACE_GOING,
            self.LAST_RACE_SURFACE,
            self.LAST_RACE_DISTANCE,
            self.LAST_RACE_WEIGHT,
            self.LAST_RACE_SPEED,
            self.DAYS_REST_SINCE_LAST_RACE,
            self.LAST_RACE_ODDS,
            self.LAST_RACE_OFFICIAL_RATING,
            self.LAST_RACE_RACING_POST_RATING,
            self.LAST_RACE_TOP_SPEED_RATING,
            self.AVG_RELATIVE_FINISHING_POSITION] + 
            [f"LastRace{surface}" for surface in surface_categories] +
            [f"LastRace{going}" for going in going_categories] +
            [f"LastRace{race_type_category}" for race_type_category in race_type_categories])

    def update(self, df : pd.DataFrame, history : pd.DataFrame, daily_slice : pd.DataFrame) -> None:
        slice_date = np.datetime64(daily_slice["Off"].min().date())
        slice_horses = daily_slice["HorseId"].unique().tolist()
        horse_history = history[history["HorseId"].isin(slice_horses)].sort_values(["HorseId", "Off"], ascending=[True, False])
        if len(horse_history) > 0:
            stats = horse_history.groupby("HorseId").apply(lambda g: self.__calculate_counts_for_race_group(slice_date, g))
            daily_stats = pd.merge(daily_slice.drop(self.new_column_names, axis=1, errors='ignore'), stats, how="left", on=["HorseId"])
            df.loc[df.index.isin(daily_slice.index), self.new_column_names] = daily_stats[self.new_column_names].values

    def __calculate_counts_for_race_group(self, current_date: np.datetime64, horse_races: pd.DataFrame) -> pd.Series:
        new_columns = {self.NUMBER_OF_PRIOR_RACES: horse_races["RaceId"].count()}
        last_race = horse_races.head(1) # Data already ordered in Off descending order
        new_columns[self.LAST_RACE_GOING] = last_race["Going"].values[0]
        new_columns[self.LAST_RACE_SURFACE] = last_race["Surface"].values[0]
        new_columns[self.LAST_RACE_DISTANCE] = last_race["DistanceInMeters"].values[0]
        new_columns[self.LAST_RACE_WEIGHT] = last_race["WeightInPounds"].values[0]
        new_columns[self.LAST_RACE_SPEED] = last_race["Speed"].values[0]
        new_columns[self.DAYS_REST_SINCE_LAST_RACE] = math.ceil((current_date - last_race["Off"].values[0]) / self.ONE_DAY)
        new_columns[self.LAST_RACE_ODDS] = last_race["DecimalOdds"].values[0]
        new_columns[self.LAST_RACE_OFFICIAL_RATING] = last_race["OfficialRating"].values[0]
        new_columns[self.LAST_RACE_RACING_POST_RATING] = last_race["RacingPostRating"].values[0]
        new_columns[self.LAST_RACE_TOP_SPEED_RATING] = last_race["TopSpeedRating"].values[0]
        new_columns[self.AVG_RELATIVE_FINISHING_POSITION] = (horse_races["FinishingPosition"] / horse_races["HorseCount"]).mean()
        for going in going_categories:
            new_columns[f"LastRace{going}"] = last_race[going].values[0]
        for surface in surface_categories:
            new_columns[f"LastRace{surface}"] = last_race[surface].values[0]
        for race_type_category in race_type_categories:
            new_columns[f"LastRace{race_type_category}"] = last_race[race_type_category].values[0]
        return pd.Series(new_columns, index=self.new_column_names) 


In [ ]:
CalculateHorsesStats().process_race_data(races)

### Evaluating previous horse performance

Now that we have calculated stats for each horse based on the previous races, we can try to figure out if any of the attributes we have relating to previous performance correlate to a horses likelihood of beating another horse in the next race. 

Possible performance attributes are:

- Racing post rating
- Official rating
- Top speed rating
- Odds 
- Average relative position (i.e. position divided by number of runners in race)



In [ ]:
def calculate_correlation_of_race_attribute(attribute: str) -> np.float64:
    head = races[["RaceId", "HorseId", "FinishingPosition", attribute]]
    cross = pd.merge(head, head, on='RaceId')
    head2head = cross[cross["HorseId_x"] > cross["HorseId_y"]].copy().dropna()
    head2head["XBeatsY"] = head2head["FinishingPosition_x"] < head2head["FinishingPosition_y"]
    head2head["Relative"] = head2head[f"{attribute}_x"] - head2head[f"{attribute}_y"]
    return head2head[["XBeatsY", "Relative"]].corr(method="spearman")["XBeatsY"]["Relative"]

In [ ]:
rp_corr = calculate_correlation_of_race_attribute("LastRaceRacingPostRating")
or_corr = calculate_correlation_of_race_attribute("LastRaceOfficialRating")
ts_corr = calculate_correlation_of_race_attribute("LastRaceTopSpeedRating")
odds_corr = calculate_correlation_of_race_attribute("LastRaceDecimalOdds")
lr_corr = calculate_correlation_of_race_attribute("LastRaceAvgRelFinishingPosition")
print(f"Correlation for Racing Post Rating {rp_corr}, Official Rating {or_corr}, Top Speed Rating {ts_corr}, Odds {odds_corr}, Avg finish {lr_corr}")

#### Conclusions

Racing post rating most strongly correlates with next race performance, other rating attributes not so much.
Official rating correlation is particularly poor. The problem with all these rating is that there are many rows where the values are undefined.

As expected odds and average finishing position are OK negative correlation indicators of prior performance (i.e. lower values correlate to finishing better). The benefits of these indicators is that they are always available for all horses.

However, all these attributes are actually relatively weak on their own (finishing position being the best with a correlation of -0.249). The predictive power is likely to be in combination with other factors. 

#### Calculate Jockey Stats

In [ ]:
class CalculateJockeyStats(RaceDataProcessor):
    NUMBER_OF_PRIOR_RACES = "JockeyNumberOfPriorRaces"
    DAYS_SINCE_LAST_RACE = "DaysSinceJockeyLastRaced"
    WIN_PERCENTAGE = "JockeyWinPercentage"
    TOP_THREE_FINISH_PERCENTAGE = "JockeyTop3Percentage"
    AVG_RELATIVE_FINISHING_POSITION = "JockeyAvgRelFinishingPosition"
    ONE_DAY = np.timedelta64(1, 'D')


    def before_process_data(self, df: pd.DataFrame) -> None:
        df.loc[:, self.NUMBER_OF_PRIOR_RACES] = 1
        self.new_column_names = [
            self.NUMBER_OF_PRIOR_RACES,
            self.DAYS_SINCE_LAST_RACE,
            self.WIN_PERCENTAGE,
            self.TOP_THREE_FINISH_PERCENTAGE,
            self.AVG_RELATIVE_FINISHING_POSITION]

    def update(self, df : pd.DataFrame, history : pd.DataFrame, daily_slice : pd.DataFrame) -> None:
        slice_jockeys = daily_slice["JockeyId"].unique().tolist()
        jockey_history = history[history["JockeyId"].isin(slice_jockeys)].sort_values(["JockeyId", "Off"], ascending=[True, False])
        if len(jockey_history) > 0:
            slice_date = np.datetime64(daily_slice["Off"].min().date())
            stats = jockey_history.groupby("JockeyId").apply(lambda g: self.__calculate_counts_for_race_group(slice_date, g))
            daily_stats = pd.merge(daily_slice.drop(self.new_column_names, axis=1, errors='ignore'), stats, how="left", on=["JockeyId"])
            df.loc[df.index.isin(daily_slice.index), self.new_column_names] = daily_stats[self.new_column_names].values

    def __calculate_counts_for_race_group(self, current_date: np.datetime64, jockey_races: pd.DataFrame) -> pd.Series:
        number_of_races = jockey_races["RaceId"].count()
        new_columns = {self.NUMBER_OF_PRIOR_RACES: number_of_races}
        last_race = jockey_races.head(1) # Data already ordered in Off descending order
        new_columns[self.DAYS_SINCE_LAST_RACE] = math.ceil((current_date - last_race["Off"].values[0]) / self.ONE_DAY)
        wins = len(jockey_races[jockey_races["FinishingPosition"] == 1])
        new_columns[self.WIN_PERCENTAGE] = wins / number_of_races
        top_finishes = len(jockey_races[jockey_races["FinishingPosition"] < 4])
        new_columns[self.TOP_THREE_FINISH_PERCENTAGE] = top_finishes / number_of_races        
        average_position = (jockey_races["FinishingPosition"] / jockey_races["HorseCount"]).mean()
        new_columns[self.AVG_RELATIVE_FINISHING_POSITION] = average_position
        return pd.Series(new_columns, index=self.new_column_names) 


In [ ]:
CalculateJockeyStats().process_race_data(races)

In [ ]:
wp_corr = calculate_correlation_of_race_attribute("JockeyWinPercentage")
pp_corr = calculate_correlation_of_race_attribute("JockeyTop3Percentage")
fp_corr = calculate_correlation_of_race_attribute("JockeyAvgRelFinishingPosition")
print(f"Correlation for Win percentage {wp_corr}, place percentage {pp_corr}, average finishing position {fp_corr}")

#### Conclusions

Win and place percentage are OK positive correlation indicator of prior performance. Average finishing position is an OK negative correlation indicator of prior performance (i.e. lower values correlate to finishing better).

However, all these attributes are actually relatively weak on there own (finishing position being the best with a correlation of -0.136). The predictive power is likely to be in combination with other factors. 

### Examples

In [ ]:
oriental_lilly_races = races[races["HorseId"] == 1439510]
oriental_lilly_races[["Off", "HorseId", "HorseName", "NumberOfPriorRaces", "FinishingPosition", "DaysRested", "LastRaceAvgRelFinishingPosition"] + [f"LastRace{going}" for going in going_categories]]

In [ ]:
# 6901 84857
james_races = races[(races["JockeyId"] == 6901)].dropna()
james_races[["Off", "JockeyId", "JockeyName", "JockeyNumberOfPriorRaces", "FinishingPosition", "HorseCount", "JockeyAvgRelFinishingPosition", "JockeyWinPercentage", "JockeyTop3Percentage"]]

### Output

In [ ]:
races.info()

In [ ]:
races.to_csv("Race_Features.csv", index=False)